# Split file into two

In [1]:
# import pandas as pd

In [2]:
# # Read original CSV
# date_parser = lambda x: datetime.strptime(x, '%Y-%m-%d')
# df_in = pd.read_csv("./raw/us-counties.csv", parse_dates=['date'], date_parser=date_parser)
# df_in.head()

In [3]:
# # Check number of rows in the original CSV
# len(df_in)

In [4]:
# # Split original CSV into 2 to keep number of rows within the maximum limit allowed in excel
# split1 = df_in[:int(len(df_in) / 2)]
# split2 = df_in[int(len(df_in) / 2):]
# print(len(split1), len(split2))

In [5]:
# # Store CSV
# split1.to_csv("./raw/us-counties-part-i.csv", index=False)
# split2.to_csv("./raw/us-counties-part-ii.csv", index=False)

# Foreign Key Generation

In [6]:
from datetime import datetime
import numpy as np
import pandas as pd

In [7]:
# Read State table to get foreign key value
df_ref = pd.read_csv("./cleaned/County.csv")
df_ref.head()

,id,county,population,state,state_id
0,1,Autauga,55601,Alabama,1
1,2,Baldwin,218022,Alabama,1
2,3,Barbour,24881,Alabama,1
3,4,Bibb,22400,Alabama,1
4,5,Blount,57840,Alabama,1


In [14]:
# Read original CSV
date_parser = lambda x: datetime.strptime(x, '%m/%d/%Y')
df_in_1 = pd.read_csv("./raw/County_covid_data_part_i.csv", parse_dates=['date'], date_parser=date_parser)
df_in_1.head()

,date,cases,deaths,county,state
0,2020-01-21,1,0,Snohomish,Washington
1,2020-01-22,1,0,Snohomish,Washington
2,2020-01-23,1,0,Snohomish,Washington
3,2020-01-24,1,0,Cook,Illinois
4,2020-01-24,1,0,Snohomish,Washington


In [15]:
# Read original CSV
date_parser = lambda x: datetime.strptime(x, '%m/%d/%Y')
df_in_2 = pd.read_csv("./raw/County_covid_data_part_ii.csv", parse_dates=['date'], date_parser=date_parser)
df_in_2.tail()

,date,cases,deaths,county,state
830249,2021-09-14,6275,53,Sweetwater,Wyoming
830250,2021-09-14,4679,11,Teton,Wyoming
830251,2021-09-14,3248,21,Uinta,Wyoming
830252,2021-09-14,1147,27,Washakie,Wyoming
830253,2021-09-14,845,7,Weston,Wyoming


In [16]:
df_in = pd.concat([df_in_1, df_in_2])
df_in.head()

,date,cases,deaths,county,state
0,2020-01-21,1,0,Snohomish,Washington
1,2020-01-22,1,0,Snohomish,Washington
2,2020-01-23,1,0,Snohomish,Washington
3,2020-01-24,1,0,Cook,Illinois
4,2020-01-24,1,0,Snohomish,Washington


In [17]:
df_in.tail()

,date,cases,deaths,county,state
830249,2021-09-14,6275,53,Sweetwater,Wyoming
830250,2021-09-14,4679,11,Teton,Wyoming
830251,2021-09-14,3248,21,Uinta,Wyoming
830252,2021-09-14,1147,27,Washakie,Wyoming
830253,2021-09-14,845,7,Weston,Wyoming


In [18]:
len(df_in)

1662816

In [19]:
# Initialize new dataframe
columns = [
    "record_date",
    "daily_positive_cases",
    "daily_deaths",
    "county",
    "state",
    "county_id"
]
df_out = pd.DataFrame([], columns=columns)
df_out.head()

,record_date,daily_positive_cases,daily_deaths,county,state,county_id


In [20]:
for state in df_in["state"].unique():
    print("[INFO] State: {}".format(state))
    df_state = df_in[df_in["state"] == state]
    for county in df_state["county"].unique():
        county_ids = df_ref[(df_ref['name'] == county) & (df_ref['state'] == state)]["id"] 
        if(len(county_ids) == 0):
            continue
        county_id = county_ids.iloc[0]
#         print("[INFO] County: {}, State: {}, County_id: {}".format(county, state, county_id))
        df_county_state = df_state[df_state["county"] == county].sort_values(by=['date'])
        for i in range(len(df_county_state)):
            current = df_county_state.iloc[i]
            date = current["date"]
            if(i == 0):
                daily_positive_cases = current["cases"]
                daily_deaths = current["deaths"]
            else:
                previous = df_county_state.iloc[i - 1]
                daily_positive_cases = current["cases"] - previous["cases"]
                daily_deaths = current["deaths"] - previous["deaths"]
            new_row = {
                "record_date": date,
                "daily_positive_cases": daily_positive_cases,
                "daily_deaths": daily_deaths,
                "county": county,
                "state": state,
                "county_id": county_id
            }
            df_out = df_out.append(new_row, ignore_index=True)

[INFO] State: Washington
[INFO] State: Illinois
[INFO] State: California
[INFO] State: Arizona
[INFO] State: Massachusetts
[INFO] State: Wisconsin
[INFO] State: Texas
[INFO] State: Nebraska
[INFO] State: Utah
[INFO] State: Oregon
[INFO] State: Florida
[INFO] State: New York
[INFO] State: Georgia
[INFO] State: New Hampshire
[INFO] State: North Carolina
[INFO] State: New Jersey
[INFO] State: Colorado
[INFO] State: Maryland
[INFO] State: Nevada
[INFO] State: Tennessee
[INFO] State: Hawaii
[INFO] State: Indiana
[INFO] State: Kentucky
[INFO] State: Minnesota
[INFO] State: Oklahoma
[INFO] State: Pennsylvania
[INFO] State: South Carolina
[INFO] State: District of Columbia
[INFO] State: Kansas
[INFO] State: Missouri
[INFO] State: Vermont
[INFO] State: Virginia
[INFO] State: Connecticut
[INFO] State: Iowa
[INFO] State: Louisiana
[INFO] State: Ohio
[INFO] State: Michigan
[INFO] State: South Dakota
[INFO] State: Arkansas
[INFO] State: Delaware
[INFO] State: Mississippi
[INFO] State: New Mexico
[I

In [22]:
df_out.head()

,record_date,daily_positive_cases,daily_deaths,county,state,county_id
0,2020-01-21,1,0,Snohomish,Washington,2982
1,2020-01-22,0,0,Snohomish,Washington,2982
2,2020-01-23,0,0,Snohomish,Washington,2982
3,2020-01-24,0,0,Snohomish,Washington,2982
4,2020-01-25,0,0,Snohomish,Washington,2982


In [23]:
df_out.to_csv("./cleaned/County_covid_data.csv", index=False)

# Split file into two

In [26]:
len(df_out)

1648737

In [27]:
df_out_1 = df_out[:len(df_out) // 2]
df_out_2 = df_out[len(df_out) // 2:]
print(len(df_out_1), len(df_out_2))

824368 824369


In [28]:
df_out_1.to_csv("./cleaned/County_covid_data_part_i.csv", index=False)
df_out_2.to_csv("./cleaned/County_covid_data_part_ii.csv", index=False)